In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
import os
import pickle
import warnings
import json
from prophet.serialize import model_to_json

warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
CONF = {
    'input_file': '/content/sales_data_final.csv', # Update path
    'forecast_horizon_weeks': 104, # Predict next 2 years
    'prophet_model_dir': 'models/prophet_individual',
    'global_model_dir': 'models/global_lgbm',
    'n_jobs': -1 # Use all cores
}

for d in [CONF['prophet_model_dir'], CONF['global_model_dir']]:
    os.makedirs(d, exist_ok=True)

# --- 1. DATA PREPROCESSING ---
def load_and_clean_data(filepath):
    print(">>> Loading Data...")
    df = pd.read_csv(filepath)

    # Identify date columns (assuming format YYYY-MM-DD or similar in cols)
    id_vars = ['Product_Code'] # Add Category if exists
    date_cols = [c for c in df.columns if c not in id_vars]

    # Melt to Long Format
    df_long = df.melt(id_vars=id_vars, value_vars=date_cols, var_name='ds', value_name='y')
    df_long['ds'] = pd.to_datetime(df_long['ds'])

    # Handle Negative/Zero Sales with Log Transform
    # log1p(x) = log(x + 1). This ensures 0 sales -> 0, and no negatives.
    df_long['y_log'] = np.log1p(df_long['y'])

    df_long = df_long.sort_values(['Product_Code', 'ds']).reset_index(drop=True)
    print(f"Data Loaded: {df_long.shape[0]} rows, {df_long['Product_Code'].nunique()} products.")
    return df_long

df_clean = load_and_clean_data(CONF['input_file'])
print(df_clean)
df_clean.to_csv('./cleaned_dataset_for_training.csv')

>>> Loading Data...
Data Loaded: 42172 rows, 811 products.
      Product_Code         ds   y     y_log
0               P1 2025-01-07  11  2.484907
1               P1 2025-01-14  12  2.564949
2               P1 2025-01-21  10  2.397895
3               P1 2025-01-28   8  2.197225
4               P1 2025-02-04  13  2.639057
...            ...        ...  ..       ...
42167          P99 2025-12-02   8  2.197225
42168          P99 2025-12-09  10  2.397895
42169          P99 2025-12-16   7  2.079442
42170          P99 2025-12-23   9  2.302585
42171          P99 2025-12-30  11  2.484907

[42172 rows x 4 columns]


In [2]:
import json
from prophet import Prophet
from prophet.serialize import model_to_json

# --- 2. PROPHET TRAINING & BASELINE GENERATION ---

def train_prophet_single(group, product_id, horizon):
    """
    Trains Prophet on log-sales.
    Returns: DataFrame containing (ds, yhat_log, Product_Code, type='history'|'future')
    """
    try:
        # Prepare data for Prophet
        df_p = group[['ds', 'y_log']].rename(columns={'y_log': 'y'})

        # Force yearly seasonality
        m = Prophet(yearly_seasonality=True,
                    weekly_seasonality=False,
                    daily_seasonality=False)

        m.fit(df_p)

        # ---------------------------------------------------------
        # CHANGE 1: Save Model as Native JSON (No Pickle)
        # ---------------------------------------------------------
        model_path = os.path.join(CONF['prophet_model_dir'], f"{product_id}.json")
        with open(model_path, 'w') as f:
            f.write(model_to_json(m))
        # ---------------------------------------------------------

        # Create Future Dataframe (History + Future)
        future = m.make_future_dataframe(periods=horizon, freq='W')
        forecast = m.predict(future)

        # Clean up result
        forecast = forecast[['ds', 'yhat']].rename(columns={'yhat': 'prophet_pred_log'})
        forecast['Product_Code'] = product_id

        # Mark rows as training or future
        max_train_date = df_p['ds'].max()
        forecast['split_type'] = forecast['ds'].apply(
            lambda x: 'train' if x <= max_train_date else 'future'
        )

        return forecast

    except Exception as e:
        print(f"Error on {product_id}: {e}")
        # Return empty DF so concat doesn't fail later
        return pd.DataFrame()

# --- Execution Block ---

print(f">>> Starting Prophet Training for {df_clean['Product_Code'].nunique()} products...")
print(f"    Forecasting Horizon: {CONF['forecast_horizon_weeks']} weeks")

products = df_clean['Product_Code'].unique()

# Run Parallel Training
results = Parallel(n_jobs=CONF['n_jobs'], verbose=5)(
    delayed(train_prophet_single)(
        df_clean[df_clean['Product_Code'] == p],
        p,
        CONF['forecast_horizon_weeks']
    ) for p in products
)

# Combine all Prophet outputs
df_prophet_full = pd.concat(results)

# ---------------------------------------------------------
# CHANGE 2: Save Forecasts as Parquet (No Pickle)
# ---------------------------------------------------------
forecast_path = os.path.join(CONF['global_model_dir'], "prophet_full_forecast.parquet")
df_prophet_full.to_parquet(forecast_path, index=False)
print(f">>> Phase 1 Complete. Baseline saved to {forecast_path}")

# ---------------------------------------------------------
# CHANGE 3: Auto-Generate Manifest from Success Files
# ---------------------------------------------------------
# Since Parallel jobs are independent, we scan the directory
# afterwards to see which models actually succeeded.
successful_models = [
    f.replace(".json", "")
    for f in os.listdir(CONF['prophet_model_dir'])
    if f.endswith(".json")
]

manifest_update = {
    "prophet_models": {
        "format": "json",
        "directory": "prophet_individual", # Ensure this matches CONF['prophet_model_dir'] relative path
        "count": len(successful_models),
        "available_items": successful_models
    }
}

# Update/Create the manifest file
manifest_path = os.path.join(CONF['global_model_dir'], "model_manifest.json")

# Load existing manifest if it exists (to keep LGBM info), else create new
if os.path.exists(manifest_path):
    with open(manifest_path, 'r') as f:
        current_manifest = json.load(f)
else:
    current_manifest = {}

current_manifest.update(manifest_update)

with open(manifest_path, 'w') as f:
    json.dump(current_manifest, f, indent=4)

print(f">>> Manifest updated with {len(successful_models)} Prophet models.")

>>> Starting Prophet Training for 811 products...
    Forecasting Horizon: 104 weeks


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  2.6min


>>> Phase 1 Complete. Baseline saved to models/global_lgbm/prophet_full_forecast.parquet
>>> Manifest updated with 811 Prophet models.


[Parallel(n_jobs=-1)]: Done 811 out of 811 | elapsed:  3.2min finished


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# --- 3. FEATURE ENGINEERING ---

def create_features(df):
    df = df.copy()

    # 1. Time Features
    df['month'] = df['ds'].dt.month
    df['week'] = df['ds'].dt.isocalendar().week.astype(int)
    df['year'] = df['ds'].dt.year

    # 2. Cyclical Encoding (Crucial for Seasonality)
    # Encodes that Month 12 is close to Month 1
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)

    return df

print(">>> Preparing LightGBM Dataset...")

# Merge Prophet Predictions with Actuals for Training
# We only want the 'train' rows from Prophet for the LightGBM training set
df_train_prophet = df_prophet_full[df_prophet_full['split_type'] == 'train']

df_ensemble = pd.merge(df_clean, df_train_prophet[['ds', 'Product_Code', 'prophet_pred_log']],
                       on=['ds', 'Product_Code'], how='inner')

# Apply Feature Engineering
df_ensemble = create_features(df_ensemble)

# Encode Product Code
le = LabelEncoder()
df_ensemble['Product_Code_Encoded'] = le.fit_transform(df_ensemble['Product_Code'])

# Create a dictionary map: {'Product_A': 0, 'Product_B': 1}
# Converting numpy ints to python ints for JSON compatibility
encoder_map = {str(k): int(v) for k, v in zip(le.classes_, le.transform(le.classes_))}

encoder_path = os.path.join(CONF['global_model_dir'], "product_encoder.json")
with open(encoder_path, 'w') as f:
    json.dump(encoder_map, f)

print(f"   Encoder mapping saved to {encoder_path}")

>>> Preparing LightGBM Dataset...
   Encoder mapping saved to models/global_lgbm/product_encoder.json


In [7]:
# --- 4. GLOBAL LIGHTGBM TRAINING ---

print(">>> Training Global LightGBM Model...")

features = [
    'prophet_pred_log',
    'Product_Code_Encoded',
    'month_sin', 'month_cos',
    'week_sin', 'week_cos',
    'year'
]
target = 'y_log' # Train on Log Sales

# Robust Time Split
# We sort by date. Last 4 weeks = Validation, Rest = Train.
unique_dates = sorted(df_ensemble['ds'].unique())
split_date = unique_dates[-4] # Reserve last 4 weeks for validation

print(f"    Splitting Train/Valid at: {split_date}")

train_mask = df_ensemble['ds'] <= split_date
valid_mask = df_ensemble['ds'] > split_date

X_train = df_ensemble.loc[train_mask, features]
y_train = df_ensemble.loc[train_mask, target]
X_valid = df_ensemble.loc[valid_mask, features]
y_valid = df_ensemble.loc[valid_mask, target]

# Create Datasets
dtrain = lgb.Dataset(X_train, label=y_train)
dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)

# Hyperparameters
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.005, # Lower learning rate for better generalization on small data
    'num_leaves': 40,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'seed': 42
}

model_lgb = lgb.train(
    params,
    dtrain,
    num_boost_round=2000,
    valid_sets=[dtrain, dvalid],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=200)
    ]
)

lgbm_filename = "lgb_global_model.txt"
lgb_model_path = os.path.join(CONF['global_model_dir'], lgbm_filename)
model_lgb.save_model(lgb_model_path)
print(f">>> LightGBM Model saved to {lgb_model_path}")

# 2. Update the Manifest Contract
manifest_path = os.path.join(CONF['global_model_dir'], "model_manifest.json")

# Load existing manifest (created in Phase 2)
if os.path.exists(manifest_path):
    with open(manifest_path, 'r') as f:
        manifest = json.load(f)
else:
    manifest = {}

# Update Global Model Section
manifest['active_global_model'] = lgbm_filename
manifest['global_model_config'] = {
    "format": "lgbm_text",
    "expected_features": len(features),
    "feature_names_ordered": features,
    "encoder_file": "product_encoder.json"
}

with open(manifest_path, 'w') as f:
    json.dump(manifest, f, indent=4)
print(">>> Manifest Contract updated.")

>>> Training Global LightGBM Model...
    Splitting Train/Valid at: 2025-12-09 00:00:00
Training until validation scores don't improve for 100 rounds
[200]	train's rmse: 0.575582	valid's rmse: 0.534947
[400]	train's rmse: 0.342573	valid's rmse: 0.316822
[600]	train's rmse: 0.287996	valid's rmse: 0.265123
[800]	train's rmse: 0.274281	valid's rmse: 0.251882
[1000]	train's rmse: 0.269795	valid's rmse: 0.247319
[1200]	train's rmse: 0.267462	valid's rmse: 0.245302
[1400]	train's rmse: 0.265826	valid's rmse: 0.244261
[1600]	train's rmse: 0.264521	valid's rmse: 0.243753
[1800]	train's rmse: 0.26331	valid's rmse: 0.24348
Early stopping, best iteration is:
[1770]	train's rmse: 0.263498	valid's rmse: 0.243418
>>> LightGBM Model saved to models/global_lgbm/lgb_global_model.txt
>>> Manifest Contract updated.


In [8]:
# NEW
print(">>> Generating Final Forecasts...")

# 1. Get the Future Prophet Data
df_future = df_prophet_full[df_prophet_full['split_type'] == 'future'].copy()

if df_future.empty:
    raise ValueError("No future dates found in Prophet predictions.")

# 2. Feature Engineering
df_future = create_features(df_future)

# 3. Encoding (Using the JSON Map we just made)
# Logic: Map known products, fill unknown with -1
df_future['Product_Code_Encoded'] = df_future['Product_Code'].map(encoder_map).fillna(-1).astype(int)

# Filter out unknown products (Safety check)
df_future = df_future[df_future['Product_Code_Encoded'] != -1]

# 4. LightGBM Prediction
print(f"    Predicting for {df_future.shape[0]} future rows...")
df_future['lgb_pred_log'] = model_lgb.predict(df_future[features])

# 5. Inverse Transformation (Log -> Normal Sales)
df_future['final_predicted_sales'] = np.expm1(df_future['lgb_pred_log']).clip(lower=0)

# 6. Format Final Output
final_submission = df_future[['ds', 'Product_Code', 'final_predicted_sales']].copy()
final_submission = final_submission.sort_values(['Product_Code', 'ds'])

output_path = 'final_sales_forecast_2years_safe.csv'
final_submission.to_csv(output_path, index=False)

print(f"✅ Pipeline Finished! Forecasts saved to: {output_path}")


>>> Generating Final Forecasts...
    Predicting for 84344 future rows...
✅ Pipeline Finished! Forecasts saved to: final_sales_forecast_2years_safe.csv


In [ ]:
# OLD --- 5. INFERENCE & FORECAST GENERATION --- Removed

>>> Generating Final Forecasts...
    Predicting for 84344 future rows...
✅ Pipeline Finished! Forecasts saved to: final_sales_forecast_2years_new.csv
           ds Product_Code  final_predicted_sales
52 2026-01-04           P1              13.034574
53 2026-01-11           P1              13.212149
54 2026-01-18           P1              10.746154
55 2026-01-25           P1               9.667746
56 2026-02-01           P1              11.313763


In [9]:
import pandas as pd
import numpy as np
from prophet import Prophet
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from sqlalchemy import create_engine
import os
import pickle
import warnings

warnings.filterwarnings('ignore')

# --- CONFIGURATION ---
CONF = {
    'input_file': '/content/sales_data_final.csv',
    'forecast_horizon_weeks': 104,  # 2 Years
    'prophet_model_dir': 'models/prophet_individual',
    'global_model_dir': 'models/global_lgbm',
    'db_connection_str': 'Local-postgres-DB-URL/sales_db', # Update this!
    'n_jobs': -1
}

for d in [CONF['prophet_model_dir'], CONF['global_model_dir']]:
    os.makedirs(d, exist_ok=True)

# --- 1. DATA PREPROCESSING ---
def load_and_clean_data(filepath):
    print(">>> Phase 1: Loading & Preprocessing...")
    df = pd.read_csv(filepath)
    id_vars = ['Product_Code']
    date_cols = [c for c in df.columns if c not in id_vars]

    df_long = df.melt(id_vars=id_vars, value_vars=date_cols, var_name='ds', value_name='y')
    df_long['ds'] = pd.to_datetime(df_long['ds'])

    # Log Transform for Training (Handle 0s and skew)
    df_long['y_log'] = np.log1p(df_long['y'])

    df_long = df_long.sort_values(['Product_Code', 'ds']).reset_index(drop=True)
    return df_long

df_clean = load_and_clean_data(CONF['input_file'])

# --- 2. PROPHET FORECAST GENERATION (HISTORY + FUTURE) ---
def train_prophet_single(group, product_id, horizon):
    try:
        df_p = group[['ds', 'y_log']].rename(columns={'y_log': 'y'})

        # Hardcoded seasonality for 52-week data
        m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
        m.fit(df_p)

        future = m.make_future_dataframe(periods=horizon, freq='W')
        forecast = m.predict(future)

        forecast = forecast[['ds', 'yhat']].rename(columns={'yhat': 'prophet_pred_log'})
        forecast['Product_Code'] = product_id

        max_train_date = df_p['ds'].max()
        forecast['split_type'] = forecast['ds'].apply(lambda x: 'train' if x <= max_train_date else 'future')

        return forecast
    except Exception as e:
        return pd.DataFrame()

print(f">>> Phase 2: Generating Prophet Baselines (Horizon: {CONF['forecast_horizon_weeks']} weeks)...")
results = Parallel(n_jobs=CONF['n_jobs'], verbose=0)(
    delayed(train_prophet_single)(df_clean[df_clean['Product_Code'] == p], p, CONF['forecast_horizon_weeks'])
    for p in df_clean['Product_Code'].unique()
)
df_prophet_full = pd.concat(results)

# --- 3. FEATURE ENGINEERING ---
def create_features(df):
    df = df.copy()
    df['month'] = df['ds'].dt.month
    df['week'] = df['ds'].dt.isocalendar().week.astype(int)
    df['year'] = df['ds'].dt.year
    # Cyclical Features
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['week_sin'] = np.sin(2 * np.pi * df['week'] / 52)
    df['week_cos'] = np.cos(2 * np.pi * df['week'] / 52)
    return df

# Merge & Prepare
df_train_prophet = df_prophet_full[df_prophet_full['split_type'] == 'train']
df_ensemble = pd.merge(df_clean, df_train_prophet[['ds', 'Product_Code', 'prophet_pred_log']],
                       on=['ds', 'Product_Code'], how='inner')
df_ensemble = create_features(df_ensemble)

le = LabelEncoder()
df_ensemble['Product_Code_Encoded'] = le.fit_transform(df_ensemble['Product_Code'])

# --- 4. LIGHTGBM TRAINING & SCORECARD ---
print(">>> Phase 3: Training Ensemble & Generating Scorecard...")

features = ['prophet_pred_log', 'Product_Code_Encoded', 'month_sin', 'month_cos', 'week_sin', 'week_cos', 'year']
target = 'y_log'

# Time-based Split (Last 4 weeks for Validation)
unique_dates = sorted(df_ensemble['ds'].unique())
split_date = unique_dates[-4]

train_mask = df_ensemble['ds'] <= split_date
valid_mask = df_ensemble['ds'] > split_date

X_train, y_train = df_ensemble.loc[train_mask, features], df_ensemble.loc[train_mask, target]
X_valid, y_valid = df_ensemble.loc[valid_mask, features], df_ensemble.loc[valid_mask, target]

# Train
model_lgb = lgb.train(
    {'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.02, 'verbose': -1},
    lgb.Dataset(X_train, label=y_train),
    num_boost_round=1000,
    valid_sets=[lgb.Dataset(X_valid, label=y_valid)],
    callbacks=[lgb.early_stopping(50)]
)

# --- SCORECARD GENERATION ---
# Predict on Validation Set
preds_log = model_lgb.predict(X_valid)
preds_real = np.expm1(preds_log).clip(min=0) # Convert back to normal scale
actuals_real = np.expm1(y_valid).values

# Calculate Metrics
mae = mean_absolute_error(actuals_real, preds_real)
mse = mean_squared_error(actuals_real, preds_real)
rmse = np.sqrt(mse)
r2 = r2_score(actuals_real, preds_real)

# WMAPE Calculation (Sum of Absolute Errors / Sum of Actuals)
wmape = np.sum(np.abs(actuals_real - preds_real)) / np.sum(actuals_real)
accuracy = 1.0 - wmape

print("\n" + "="*40)
print("       MODEL PERFORMANCE SCORECARD       ")
print("="*40)
print(f" Dataset Split Date : {split_date}")
print("-" * 40)
print(f" R-Squared (R²)     : {r2:.4f}  (Fit Quality)")
print(f" MAE                : {mae:.2f}  (Avg Error per unit)")
print(f" RMSE (MMSE Proxy)  : {rmse:.2f}  (Penalizes large errors)")
print(f" WMAPE              : {wmape:.2%}")
print("-" * 40)
print(f" >> ACCURACY        : {accuracy:.2%} <<")
print("="*40 + "\n")

>>> Phase 1: Loading & Preprocessing...
>>> Phase 2: Generating Prophet Baselines (Horizon: 104 weeks)...
>>> Phase 3: Training Ensemble & Generating Scorecard...
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[315]	valid_0's rmse: 0.239147

       MODEL PERFORMANCE SCORECARD       
 Dataset Split Date : 2025-12-09 00:00:00
----------------------------------------
 R-Squared (R²)     : 0.9540  (Fit Quality)
 MAE                : 1.34  (Avg Error per unit)
 RMSE (MMSE Proxy)  : 2.22  (Penalizes large errors)
 WMAPE              : 15.11%
----------------------------------------
 >> ACCURACY        : 84.89% <<



In [10]:

# --- 5. FINAL FORECAST & DB EXPORT ---
print(">>> Phase 4: Production Forecast & Database Export...")

# Prepare Future Data
df_future = df_prophet_full[df_prophet_full['split_type'] == 'future'].copy()
df_future = create_features(df_future)
df_future['Product_Code_Encoded'] = df_future['Product_Code'].apply(
    lambda x: le.transform([x])[0] if x in le.classes_ else -1
)
df_future = df_future[df_future['Product_Code_Encoded'] != -1]

# Predict
df_future['predicted_log'] = model_lgb.predict(df_future[features])
df_future['predicted_sales'] = np.expm1(df_future['predicted_log']).clip(lower=0)

# Format for DB (Clean Columns)
df_export = df_future[['ds', 'Product_Code', 'predicted_sales']].rename(
    columns={'ds': 'forecast_date', 'Product_Code': 'product_code'}
)

>>> Phase 4: Production Forecast & Database Export...


In [12]:
# --- 6. SAVE TO DB USING PRISMA  ---
import asyncio
print(">>> Phase 6: Saving Forecasts and Metrics to DB (Prisma)...")

# 1. Update function to ACCEPT the data as arguments
async def save_results_to_db(metrics, df_forecasts):
    # Connect
    db = Prisma()
    await db.connect()

    try:
        # --- A. SAVE METRICS ---
        print(f"   -> Saving Metrics (Accuracy: {metrics['accuracy']:.2%})...")

        await db.modelmetric.create(data={
            'training_run_date': pd.Timestamp.now().to_pydatetime(), # Safe conversion
            'model_version': 'Hybrid-Prophet-LGBM-v1',
            'wmape': float(metrics['wmape']),
            'accuracy': float(metrics['accuracy']),
            'rmse': float(metrics['rmse']),
            'mae': float(metrics['mae']),
            'description': 'Automated Weekly Retraining'
        })

        # --- B. SAVE FORECASTS ---
        print("   -> Saving Forecasts...")

        forecast_records = []
        for _, row in df_forecasts.iterrows():
            forecast_records.append({
                'product_code': str(row['product_code']),
                'forecast_date': row['forecast_date'],
                'predicted_sales': float(row['predicted_sales'])
            })

        await db.salesforecast.delete_many()

        await db.salesforecast.create_many(
            data=forecast_records,
            skip_duplicates=True
        )

        print(f"✅ Success! Saved {len(forecast_records)} forecasts and metrics.")

    except Exception as e:
        print(f"❌ DB Error: {e}")
        import traceback
        traceback.print_exc() # Print full error details

    finally:
        await db.disconnect()

# --- 2. EXECUTE WITH ARGUMENTS ---
if __name__ == "__main__":

    # Ensure these variables exist in your main scope!
    # If they are inside a function, return them first.
    current_metrics = {
        'wmape': wmape,
        'accuracy': accuracy,
        'rmse': rmse,
        'mae': mae
    }

    asyncio.run(save_results_to_db(current_metrics, df_export))

>>> Phase 6: Saving Forecasts and Metrics to DB (Prisma)...


RuntimeError: asyncio.run() cannot be called from a running event loop

In [13]:
# Also save CSV as backup
df_export.to_csv('final_forecast_backup.csv', index=False)

In [14]:
model_lgb.save_model('./trained_lgb_model_for_metrics_new.txt')

In [15]:
df_ensemble.to_csv('./latest_df_ensemble.csv')